-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# LLMOps
In this example, we will walk through some key steps for taking an LLM-based pipeline to production.  Our pipeline will be familiar to you from previous modules: summarization of news articles using a pre-trained model from Hugging Face.  But in this walkthrough, we will be more rigorous about LLMOps.

**Develop an LLM pipeline**

Our LLMOps goals during development are (a) to track what we do carefully for later auditing and reproducibility and (b) to package models or pipelines in a format which will make future deployment easier.  Step-by-step, we will:
* Load data.
* Build an LLM pipeline.
* Test applying the pipeline to data, and log queries and results to MLflow Tracking.
* Log the pipeline to the MLflow Tracking server as an MLflow Model.

**Test the LLM pipeline**

Our LLMOps goals during testing (in the staging or QA stage) are (a) to track the LLM's progress through testing and towards production and (b) to do so programmatically to demonstrate the APIs needed for future CI/CD automation.  Step-by-step, we will:
* Register the pipeline to the MLflow Model Registry.
* Test the pipeline on sample data.
* Promote the registered model (pipeline) to production.

**Create a production workflow for batch inference**

Our LLMOps goals during production are (a) to write scale-out code which can meet scaling demands in the future and (b) to simplify deployment by using MLflow to write model-agnostic deployment code.  Step-by-step, we will:
* Load the latest production LLM pipeline from the Model Registry.
* Apply the pipeline to an Apache Spark DataFrame.
* Append the results to a Delta Lake table.

### Notes about this workflow

**This notebook vs. modular scripts**: Since this demo is in a single notebook, we will divide the workflow from development to production via notebook sections.  In a more realistic LLM Ops setup, you would likely have the sections split into separate notebooks or scripts.

**Promoting models vs. code**: We track the path from development to production via the MLflow Model Registry.  That is, we are *promoting models* towards production, rather than promoting code.  For more discussion of these two paradigms, see ["The Big Book of MLOps"](https://www.databricks.com/resources/ebook/the-big-book-of-mlops).

### ![Dolly](https://files.training.databricks.com/images/llm/dolly_small.png) Learning Objectives
1. Walk through a simple but realistic workflow to take an LLM pipeline from development to production.
1. Make use of MLflow Tracking and the Model Registry to package and manage the pipeline.
1. Scale out batch inference using Apache Spark and Delta Lake.

## Classroom Setup

In [0]:
%run ../Includes/Classroom-Setup

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


Resetting the learning environment:
| enumerating serving endpoints...found 0...(0 seconds)
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/large-language-models/v01"


Importing lab testing framework.



Using the "default" schema.

Predefined paths variables:
| DA.paths.working_dir: /dbfs/mnt/dbacademy-users/vijaymohire@bhadaleit.onmicrosoft.com/large-language-models
| DA.paths.user_db:     /dbfs/mnt/dbacademy-users/vijaymohire@bhadaleit.onmicrosoft.com/large-language-models/database.db
| DA.paths.datasets:    /dbfs/mnt/dbacademy-datasets/large-language-models/v01

Setup completed (5 seconds)

The models developed or used in this course are for demonstration and learning purposes only.
Models may occasionally output offensive, inaccurate, biased information, or harmful instructions.


For this notebook we'll use the <a href="https://huggingface.co/datasets/xsum" target="_blank">Extreme Summarization (XSum) Dataset</a>  with the <a href="https://huggingface.co/t5-small" target="_blank">T5 Text-To-Text Transfer Transformer</a> from Hugging Face.

## Prepare data

In [0]:
from datasets import load_dataset
from transformers import pipeline

In [0]:
xsum_dataset = load_dataset(
    "xsum", version="1.2.0", cache_dir="/dbfs/mnt/dbacademy-datasets/large-language-models/v01"
)  # Note: We specify cache_dir to use pre-cached data.
xsum_sample = xsum_dataset["train"].select(range(10))
display(xsum_sample.to_pandas())

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


Found cached dataset xsum (/dbfs/mnt/dbacademy-datasets/large-language-models/v01/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

document summary id The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we're neglected or forgotten," she said.
"That may not be true but it is perhaps my perspective over the last few days.
"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"
Meanwhile, a flood alert remains in place across the Borders because of the constant rain.
Peebles was badly hit by problems, sparking calls to introduce more defences in the area.
Scottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.
The Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.
He said it was important to get the flood protection plan right but backed calls to speed up the process.
"I was quite taken aback by the amount of damage that has been done," he said.
"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."
He said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.
Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk. Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank. 35232142 A fire alarm went off at the Holiday Inn in Hope Street at about 04:20 BST on Saturday and guests were asked to leave the hotel.
As they gathered outside they saw the two buses, parked side-by-side in the car park, engulfed by flames.
One of the tour groups is from Germany, the other from China and Taiwan. It was their first night in Northern Ireland.
The driver of one of the buses said many of the passengers had left personal belongings on board and these had been destroyed.
Both groups have organised replacement coaches and will begin their tour of the north coast later than they had planned.
Police have appealed for information about the attack.
Insp David Gibson said: "It appears as though the fire started under one of the buses before spreading to the second.
"While the exact cause is still under investigation, it is thought that the fire was started deliberately." Two tourist buses have been destroyed by fire in a suspected arson attack in Belfast city centre. 40143035 Ferrari appeared in a position to challenge until the final laps, when the Mercedes stretched their legs to go half a second clear of the red cars.
Sebastian Vettel will start third ahead of team-mate Kimi Raikkonen.
The world champion subsequently escaped punishment for reversing in the pit lane, which could have seen him stripped of pole.
But stewards only handed Hamilton a reprimand, after governing body the FIA said "no clear instruction was given on where he should park".
Belgian Stoffel Vandoorne out-qualified McLaren 

Later on, when we show Production inference, we will want a dataset saved for it.  See the production section below for more information about Delta, the format we use to save the data here.

In [0]:
prod_data_path = f"/dbfs/mnt/dbacademy-datasets/large-language-models/v01/m6_prod_data"
test_spark_dataset = spark.createDataFrame(xsum_dataset["test"].to_pandas())
test_spark_dataset.write.format("delta").mode("overwrite").save(prod_data_path)

## Develop an LLM pipeline

### Create a Hugging Face pipeline

In [0]:
from transformers import pipeline

# Later, we plan to log all of these parameters to MLflow.
# Storing them as variables here will help with that.
hf_model_name = "t5-small"
min_length = 20
max_length = 40
truncation = True
do_sample = True

summarizer = pipeline(
    task="summarization",
    model=hf_model_name,
    min_length=min_length,
    max_length=max_length,
    truncation=truncation,
    do_sample=do_sample,
    model_kwargs={"cache_dir": "/dbfs/mnt/dbacademy-datasets/large-language-models/v01"},
)  # Note: We specify cache_dir to use pre-cached models.

We can now examine the `summarizer` pipeline summarizing a document from the `xsum` dataset.

In [0]:
doc0 = xsum_sample["document"][0]
print(f"Summary: {summarizer(doc0)[0]['summary_text']}")
print("===============================================")
print(f"Original Document: {doc0}")

/databricks/python/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Summary: the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . a flood alert remains in place across the
Original Document: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative 

### Track LLM development with MLflow

[MLflow](https://mlflow.org/) has a Tracking component that helps you to track exactly how models or pipelines are produced during development.  Although we are not fitting (tuning or training) a model here, we can still make use of tracking to:
* Track example queries and responses to the LLM pipeline, for later review or analysis
* Store the model as an [MLflow Model flavor](https://mlflow.org/docs/latest/models.html#built-in-model-flavors), thus packaging it for simpler deployment

In [0]:
# Apply to a batch of articles
import pandas as pd

results = summarizer(xsum_sample["document"])
display(pd.DataFrame(results, columns=["summary_text"]))

summary_text
"many businesses and householders were affected by flooding in Newton Stewart . the water breached a retaining wall, flooding many commercial properties . a flood alert remains in place across"
fire alarm went off at the Holiday Inn in Hope Street on Saturday . guests were asked to leave the hotel as they gathered outside . both groups have organised replacement coaches and will begin
"stewards only handed Hamilton a reprimand after governing body said ""no clear instruction was given on where he should park"" Stoffel Vandoorne"
"the 67-year-old is accused of committing the offences between March 1972 and October 1989 . he denies all the charges, including two counts of indecency"
a man receiving treatment at the clinic threatened to shoot himself and others . the incident comes amid tension in Istanbul following several attacks in crowded areas .
Gregor Townsend gave a debut to powerhouse wing Taqele Naiyaravoro . the dragons gave first starts of the season to wingaby
"Veronica Vanessa Chango-Alverez, 31, was killed and another man injured in the crash . police want to trace Nathan Davis, 27, who has links to the Audi ."
the 25-year-old was hit by a motorbike during a race in Gent-Wevelgem . he was riding for the Wanty-Gobert team and
gundogan will not be fit for the start of the premier league season at Brighton on 12 august . the german missed the 2014 world cup after a dislocated kneecap .
"the crash happened about 07:20 GMT at the junction of the A127 and Progress Road in leigh-on-Sea, Essex . the man, aged in his 20s"


[MLflow Tracking](https://mlflow.org/docs/latest/tracking.html) is organized hierarchically as follows:
* **An [experiment](https://mlflow.org/docs/latest/tracking.html#organizing-runs-in-experiments)** generally corresponds to the creation of 1 primary model or pipeline.  In our case, this is our LLM pipeline.  It contains some number of *runs*.
   * **A [run](https://mlflow.org/docs/latest/tracking.html#organizing-runs-in-experiments)** generally corresponds to the creation of 1 sub-model, such as 1 trial during hyperparameter tuning in traditional ML.  In our case, executing this notebook once will only create 1 run, but a second execution of the notebook will create a second run.  This version tracking can be useful during iterative development.  Each run contains some number of logged parameters, metrics, tags, models, artifacts, and other metadata.
      * **A [parameter](https://mlflow.org/docs/latest/tracking.html#concepts)** is an input to the model or pipeline, such as a regularization parameter in traditional ML or `max_length` for our LLM pipeline.
      * **A [metric](https://mlflow.org/docs/latest/tracking.html#concepts)** is an output of evaluation, such as accuracy or loss.
      * **An [artifact](https://mlflow.org/docs/latest/tracking.html#concepts)** is an arbitrary file stored alongside a run's metadata, such as the serialized model itself.
      * **A [flavor](https://mlflow.org/docs/latest/models.html#storage-format)** is an MLflow format for serializing models.  This format uses the underlying ML library's format (such as PyTorch, TensorFlow, Hugging Face, or your custom format), plus metadata.

MLflow has an API for tracking queries and predictions [`mlflow.llm.log_predictions()`](https://mlflow.org/docs/latest/python_api/mlflow.llm.html), which we will use below.  Note that, as of MLflow 2.3.1 (Apr 28, 2023), this API is Experimental, so it may change in later releases.  See the [LLM Tracking page](https://mlflow.org/docs/latest/llm-tracking.html) for more information.

***Tip***: We wrap our model development workflow with a call to `with mlflow.start_run():`.  This context manager syntax starts and ends the MLflow run explicitly, which is a best practice for code which may be moved to production.  See the [API doc](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.start_run) for more information.

In [0]:
import mlflow

# Tell MLflow Tracking to user this explicit experiment path,
# which is in your home directory under the Workspace browser (left-hand sidebar).
mlflow.set_experiment(f"/Users/vijaymohire@bhadaleit.onmicrosoft.com/LLM 06 - MLflow experiment")

with mlflow.start_run():
    # LOG PARAMS
    mlflow.log_params(
        {
            "hf_model_name": hf_model_name,
            "min_length": min_length,
            "max_length": max_length,
            "truncation": truncation,
            "do_sample": do_sample,
        }
    )

    # --------------------------------
    # LOG INPUTS (QUERIES) AND OUTPUTS
    # Logged `inputs` are expected to be a list of str, or a list of str->str dicts.
    results_list = [r["summary_text"] for r in results]

    # Our LLM pipeline does not have prompts separate from inputs, so we do not log any prompts.
    mlflow.llm.log_predictions(
        inputs=xsum_sample["document"],
        outputs=results_list,
        prompts=["" for _ in results_list],
    )

    # ---------
    # LOG MODEL
    # We next log our LLM pipeline as an MLflow model.
    # This packages the model with useful metadata, such as the library versions used to create it.
    # This metadata makes it much easier to deploy the model downstream.
    # Under the hood, the model format is simply the ML library's native format (Hugging Face for us), plus metadata.

    # It is valuable to log a "signature" with the model telling MLflow the input and output schema for the model.
    signature = mlflow.models.infer_signature(
        xsum_sample["document"][0],
        mlflow.transformers.generate_signature_output(
            summarizer, xsum_sample["document"][0]
        ),
    )
    print(f"Signature:\n{signature}\n")

    # For mlflow.transformers, if there are inference-time configurations,
    # those need to be saved specially in the log_model call (below).
    # This ensures that the pipeline will use these same configurations when re-loaded.
    inference_config = {
        "min_length": min_length,
        "max_length": max_length,
        "truncation": truncation,
        "do_sample": do_sample,
    }

    # Logging a model returns a handle `model_info` to the model metadata in the tracking server.
    # This `model_info` will be useful later in the notebook to retrieve the logged model.
    model_info = mlflow.transformers.log_model(
        transformers_model=summarizer,
        artifact_path="summarizer",
        task="summarization",
        inference_config=inference_config,
        signature=signature,
        input_example="This is an example of a long news article which this pipeline can summarize for you.",
    )

2023/08/17 07:26:39 INFO mlflow.tracking.fluent: Experiment with name '/Users/vijaymohire@bhadaleit.onmicrosoft.com/LLM 06 - MLflow experiment' does not exist. Creating a new experiment.
2023/08/17 07:26:42 INFO mlflow.tracking.llm_utils: Creating a new llm_predictions.csv for run 500e4f8b05d747be9b099c6959579818.


Signature:
inputs: 
  [string]
outputs: 
  [string]




2023/08/17 07:26:45 WARNING mlflow.utils.requirements_utils: Found torch version (1.13.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==1.13.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/08/17 07:26:45 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.14.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torchvision==0.14.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### Query the MLflow Tracking server

**MLflow Tracking API**: We briefly show how to query the logged model and metadata in the MLflow Tracking server, by loading the logged model.  See the [MLflow API](https://mlflow.org/docs/latest/python_api/mlflow.html) for more information about programmatic access.

**MLflow Tracking UI**: You can also use the UI.  In the right-hand sidebar, click the beaker icon to access the MLflow experiments run list, and then click through to access the Tracking server UI.  There, you can see the logged metadata and model.  Note in particular that our LLM inputs and outputs have been logged as a CSV file under model artifacts.

GIF of MLflow UI:
![GIF of MLflow UI](https://files.training.databricks.com/images/llm/llmops.gif)

Now, we can load the pipeline back from MLflow as a [pyfunc](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html) and use the `.predict()` method to summarize an example document.

In [0]:
loaded_summarizer = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)
loaded_summarizer.predict(xsum_sample["document"][0])

'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . a flood alert remains in place across the'

The `.predict()` method can handle more than one document at a time, below we pass in all the data from `xsum_sample`.

In [0]:
results = loaded_summarizer.predict(xsum_sample.to_pandas()["document"])
display(pd.DataFrame(results, columns=["generated_summary"]))

generated_summary
the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining
fire alarm went off at the Holiday Inn in Hope Street on Saturday . guests were asked to leave the hotel . one of the buses said many of the passengers had left personal belongings
Sebastian Vettel will start third ahead of team-mate Kimi Raikkonen . stewards only handed Hamilton a reprimand after governing body said no instruction
"the 67-year-old is accused of committing the offences between March 1972 and October 1989 . he denies all the charges, including two counts of indecency"
a man receiving treatment at the clinic threatened to shoot himself and others . the incident comes amid tension in Istanbul following several attacks in crowded areas .
Gregor Townsend gave a debut to powerhouse wing Taqele Naiyaravoro . the dragons gave first starts of the season to wing a
"Veronica Vanessa Chango-Alverez, 31, was killed and another man injured in the crash . police want to trace Nathan Davis, 27, who has links to the Audi ."
the 25-year-old was hit by a motorbike during the Gent-Wevelgem race . the race passed through northern France . it was the first time a
"german will not be fit for the start of the premier league season at Brighton on 12 august . the 26-year-old says his recovery time is now being measured in ""weeks"""
"the crash happened about 07:20 GMT at the junction of the A127 and Progress Road in leigh-on-Sea, Essex . the man, aged in his 20s"


We are now ready to move to the staging step of deployment.  To get started, we will register the model in the MLflow Model Registry (more info below).

In [0]:
# Define the name for the model in the Model Registry.
# We filter out some special characters which cannot be used in model names.
model_name = f"summarizer - vijaymohire@bhadaleit.onmicrosoft.com"
model_name = model_name.replace("/", "_").replace(".", "_").replace(":", "_")
print(model_name)

summarizer - vijaymohire@bhadaleit_onmicrosoft_com


In [0]:
# Register a new model under the given name, or a new model version if the name exists already.
mlflow.register_model(model_uri=model_info.model_uri, name=model_name)

Successfully registered model 'summarizer - vijaymohire@bhadaleit_onmicrosoft_com'.
2023/08/17 07:44:09 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: summarizer - vijaymohire@bhadaleit_onmicrosoft_com, version 1
Created version '1' of model 'summarizer - vijaymohire@bhadaleit_onmicrosoft_com'.


<ModelVersion: aliases=[], creation_timestamp=1692258249408, current_stage='None', description='', last_updated_timestamp=1692258268494, name='summarizer - vijaymohire@bhadaleit_onmicrosoft_com', run_id='500e4f8b05d747be9b099c6959579818', run_link='', source='dbfs:/databricks/mlflow-tracking/1171692323008418/500e4f8b05d747be9b099c6959579818/artifacts/summarizer', status='READY', status_message='', tags={}, user_id='vijaymohire@bhadaleit.onmicrosoft.com', version='1'>

## Test the LLM pipeline

During the Staging step of development, our goal is to move code and/or models from Development to Production.  In order to do so, we must test the code and/or models to make sure they are ready for Production.

We track our progress here using the [MLflow Model Registry](https://mlflow.org/docs/latest/model-registry.html).  This metadata and model store organizes models as follows:
* **A registered model** is a named model in the registry, in our case corresponding to our summarization model.  It may have multiple *versions*.
   * **A model version** is an instance of a given model.  As you update your model, you will create new versions.  Each version is designated as being in a particular *stage* of deployment.
      * **A stage** is a stage of deployment: `None` (development), `Staging`, `Production`, or `Archived`.

The model we registered above starts with 1 version in stage `None` (development).

In the workflow below, we will programmatically transition the model from development to staging to production.  For more information on the Model Registry API, see the [Model Registry docs](https://mlflow.org/docs/latest/model-registry.html).  Alternatively, you can edit the registry and make model stage transitions via the UI.  To access the UI, click the Experiments menu option in the left-hand sidebar, and search for your model name.

In [0]:
from mlflow import MlflowClient

client = MlflowClient()

In [0]:
client.search_registered_models(filter_string=f"name = '{model_name}'")

[<RegisteredModel: aliases={}, creation_timestamp=1692258249096, description='', last_updated_timestamp=1692258249408, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1692258249408, current_stage='None', description='', last_updated_timestamp=1692258268494, name='summarizer - vijaymohire@bhadaleit_onmicrosoft_com', run_id='500e4f8b05d747be9b099c6959579818', run_link='', source='dbfs:/databricks/mlflow-tracking/1171692323008418/500e4f8b05d747be9b099c6959579818/artifacts/summarizer', status='READY', status_message='', tags={}, user_id='vijaymohire@bhadaleit.onmicrosoft.com', version='1'>], name='summarizer - vijaymohire@bhadaleit_onmicrosoft_com', tags={}>]

In the metadata above, you can see that the model is currently in stage `None` (development).  In this workflow, we will run manual tests, but it would be reasonable to run both automated evaluation and human evaluation in practice.  Once tests pass, we will promote the model to stage `Production` to mark it ready for user-facing applications.

*Model URIs*: Below, we use model URIs to tell MLflow which model and version we are referring to.  Two common URI patterns for the MLflow Model Registry are:
* `f"models:/{model_name}/{model_version}"` to refer to a specific model version by number
* `f"models:/{model_name}/{model_stage}"` to refer to the latest model version in a given stage

In [0]:
model_version = 1
dev_model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
dev_model

mlflow.pyfunc.loaded_model:
  artifact_path: summarizer
  flavor: mlflow.transformers
  run_id: 500e4f8b05d747be9b099c6959579818

*Note about model dependencies*:
When you load the model via MLflow above, you may see warnings about the Python environment.  It is very important to ensure that the environments for development, staging, and production match.
* For this demo notebook, everything is done within the same notebook environment, so we do not need to worry about libraries and versions.  However, in the Production section below, we demonstrate how to pass the `env_manager` argument to the method for loading the saved MLflow model, which tells MLflow what tooling to use to recreate the environment.
* To create a genuine production job, make sure to install the needed libraries.  MLflow saves these libraries and versions alongside the logged model; see the [MLflow docs on model storage](https://mlflow.org/docs/latest/models.html#storage-format) for more information.  While using Databricks for this course, you can also generate an example inference notebook which includes code for setting up the environment; see [the model inference docs](https://docs.databricks.com/machine-learning/manage-model-lifecycle/index.html#use-model-for-inference) for batch or streaming inference for more information.

### Transition to Staging

We will move the model to stage `Staging` to indicate that we are actively testing it.

In [0]:
client.transition_model_version_stage(model_name, model_version, "staging")

<ModelVersion: aliases=[], creation_timestamp=1692258249408, current_stage='Staging', description='', last_updated_timestamp=1692258374406, name='summarizer - vijaymohire@bhadaleit_onmicrosoft_com', run_id='500e4f8b05d747be9b099c6959579818', run_link='', source='dbfs:/databricks/mlflow-tracking/1171692323008418/500e4f8b05d747be9b099c6959579818/artifacts/summarizer', status='READY', status_message='', tags={}, user_id='7959437289808858', version='1'>

In [0]:
staging_model = dev_model

# An actual CI/CD workflow might load the `staging_model` programmatically.  For example:
#   mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{Staging}")
# or
#   mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

We now "test" the model manually on sample data. Here, we simply print out results and compare them with the original data.  In a more realistic setting, we might use a set of human evaluators to decide whether the model outperformed the previous model or system.

In [0]:
results = staging_model.predict(xsum_sample.to_pandas()["document"])
display(pd.DataFrame(results, columns=["generated_summary"]))

generated_summary
"the full cost of damage in Newton Stewart is still being assessed . the water breached a retaining wall, flooding many commercial properties . a flood alert remains in place across"
fire alarm went off at the Holiday Inn in Hope Street on Saturday . guests were asked to leave the hotel . the two buses were parked side-by-side in the car
"stewards only handed Hamilton a reprimand after FIA said ""no clear instruction was given on where he should park"" Stoffel Vandoorne"
"the 67-year-old is accused of committing the offences between March 1972 and October 1989 . he denies all the charges, including two counts of indecency"
a man receiving psychiatric treatment at the clinic threatened to shoot himself and others . the incident comes amid tension in Istanbul following several attacks in crowded areas .
Gregor Townsend gave a debut to powerhouse wing Taqele Naiyaravoro . the dragons gave first starts of the season to wing a
"Veronica Vanessa Chango-Alverez, 31, was killed and another man injured in the crash . police want to trace Nathan Davis, 27, who has links to the Audi ."
the 25-year-old was hit by a motorbike during the Gent-Wevelgem race . he was riding for the Wanty-Gobert team and was taken
"the 26-year-old says he ""can see the finishing line"" after tearing cruciate knee ligaments . he will not be fit for the start of the premier"
"the crash happened about 07:20 GMT at the junction of the A127 and Progress Road in leigh-on-Sea, Essex . the man, aged in his 20s"


### Transition to Production

The results look great!  :) Let's transition the model to Production.

In [0]:
client.transition_model_version_stage(model_name, model_version, "production")

<ModelVersion: aliases=[], creation_timestamp=1692258249408, current_stage='Production', description='', last_updated_timestamp=1692258439250, name='summarizer - vijaymohire@bhadaleit_onmicrosoft_com', run_id='500e4f8b05d747be9b099c6959579818', run_link='', source='dbfs:/databricks/mlflow-tracking/1171692323008418/500e4f8b05d747be9b099c6959579818/artifacts/summarizer', status='READY', status_message='', tags={}, user_id='7959437289808858', version='1'>

## Create a production workflow for batch inference

Once the LLM pipeline is in Production, it may be used by one or more production jobs or serving endpoints.  Common deployment locations are:
* Batch or streaming inference jobs
* Model serving endpoints
* Edge devices

Here, we will show batch inference using Apache Spark DataFrames, with Delta Lake format.  Spark allows simple scale-out inference for high-throughput, low-cost jobs, and Delta allows us to append to and modify inference result tables with ACID transactions.  See the [Apache Spark page](https://spark.apache.org/) and the [Delta Lake page](https://delta.io/) more more information on these technologies.

In [0]:
# Load our data as a Spark DataFrame.
# Recall that we saved this as Delta at the start of the notebook.
# Also note that it has a ground-truth summary column.
prod_data = spark.read.format("delta").load(prod_data_path).limit(10)
display(prod_data)

document summary id Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.
However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.
Andrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need for accommodation was "chronic".
"There's a desperate need for it, finding suitable accommodation for those leaving prison there is just a lack of it everywhere," he said.
"It could take six months to a year, without a lot of help they could be on the streets for six months.
"When you think of the consequences of either being on the street, especially with the cold weather at the moment or you may have a roof over your head, sometimes there is only one choice."
Mr Stevens believes building more one-bedroom flats could help ease the problem.
"The average price is a hundred pounds a week to keep someone in a rented flat, prison is a lot more than that so I would imagine it would save the public purse quite a few pounds," he said.
Official figures show 830 one-bedroom properties were built in the year to March 2016, of an overall total of 6,900 new properties in Wales.
Marc, 50, who has been in and out of prison for the past 20 years for burglary offences, said he struggled to find accommodation each time he was released.
He said he would ask himself: "Where am I going to stay? Where am I going to live? Have I got somewhere where I can see my daughter."
"You're put out among the same sort of people doing the same sort of thing, and it's difficult, it's difficult to get away from it. It's like every man for himself, there's nothing."
Marc has now found stable accommodation with homeless charity Emmaus and said it had been life changing.
"You feel safe, you got hot food, you've got company of people in similar situations to yourself but all dealing with different issues. It's a constructive, helpful atmosphere," he said.
Tom Clarke, chief executive of Emmaus South Wales, agreed there was not enough support available.
"We do still see [people] homeless on the streets, so clearly they haven't got accommodation and haven't got provision," he said.
"I think the key is connecting people with the services they need. I don't delude myself that Emmaus can offer a one size fits all for everyone, we can't.
"But there must be other opportunities and given suitable encouragement I believe that can and should happen."
A Welsh Government spokesman said the national pathway for homeless services to children, young people and adults in the secure estate had prevented many people from losing their home whilst serving their prison sentence.
It added there were already significant demands for one-bedroom flats across the public and private sector and it was providing 20,000 new affordable homes in the next five years. There is a "chronic" need for more housing for prison leavers in Wales, according to a charity. 38264402 Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday.
Detectives said three firearms, ammunition and a five-figure sum of money were recovered.
A 26-year-old man who was arrested and charged appeared at Edinburgh Sheriff Court on Thursday. A man has appeared in court after firearms, ammunition and ca

Below, we load the model using `mlflow.pyfunc.spark_udf`.  This returns the model as a Spark User Defined Function which can be applied efficiently to big data.  *Note that the deployment code is library-agnostic: it never references that the model is a Hugging Face pipeline.*  This simplified deployment is possible because MLflow logs environment metadata and "knows" how to load the model and run it.

In [0]:
# MLflow lets you grab the latest model version in a given stage.  Here, we grab the latest Production version.
prod_model_udf = mlflow.pyfunc.spark_udf(
    spark,
    model_uri=f"models:/{model_name}/Production",
    env_manager="local",
    result_type="string",
)

2023/08/17 07:48:14 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.
2023/08/17 07:48:15 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


In [0]:
# Run inference by appending a new column to the DataFrame

batch_inference_results = prod_data.withColumn(
    "generated_summary", prod_model_udf("document")
)
display(batch_inference_results)

document summary id generated_summary Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.
However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.
Andrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need for accommodation was "chronic".
"There's a desperate need for it, finding suitable accommodation for those leaving prison there is just a lack of it everywhere," he said.
"It could take six months to a year, without a lot of help they could be on the streets for six months.
"When you think of the consequences of either being on the street, especially with the cold weather at the moment or you may have a roof over your head, sometimes there is only one choice."
Mr Stevens believes building more one-bedroom flats could help ease the problem.
"The average price is a hundred pounds a week to keep someone in a rented flat, prison is a lot more than that so I would imagine it would save the public purse quite a few pounds," he said.
Official figures show 830 one-bedroom properties were built in the year to March 2016, of an overall total of 6,900 new properties in Wales.
Marc, 50, who has been in and out of prison for the past 20 years for burglary offences, said he struggled to find accommodation each time he was released.
He said he would ask himself: "Where am I going to stay? Where am I going to live? Have I got somewhere where I can see my daughter."
"You're put out among the same sort of people doing the same sort of thing, and it's difficult, it's difficult to get away from it. It's like every man for himself, there's nothing."
Marc has now found stable accommodation with homeless charity Emmaus and said it had been life changing.
"You feel safe, you got hot food, you've got company of people in similar situations to yourself but all dealing with different issues. It's a constructive, helpful atmosphere," he said.
Tom Clarke, chief executive of Emmaus South Wales, agreed there was not enough support available.
"We do still see [people] homeless on the streets, so clearly they haven't got accommodation and haven't got provision," he said.
"I think the key is connecting people with the services they need. I don't delude myself that Emmaus can offer a one size fits all for everyone, we can't.
"But there must be other opportunities and given suitable encouragement I believe that can and should happen."
A Welsh Government spokesman said the national pathway for homeless services to children, young people and adults in the secure estate had prevented many people from losing their home whilst serving their prison sentence.
It added there were already significant demands for one-bedroom flats across the public and private sector and it was providing 20,000 new affordable homes in the next five years. There is a "chronic" need for more housing for prison leavers in Wales, according to a charity. 38264402 prison link Cymru says ex-offenders living rough for up to a year before finding suitable accommodation . change to the Housing Act in Wales removed the right for prison leaver Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday.
Detectives said three firearms, ammunition and a fiv

We can now write out our inference results to another Delta table.  Here, we append the results to an existing table (and create the table if it does not exist).

In [0]:
inference_results_path = f"/dbfs/mnt/dbacademy-users/vijaymohire@bhadaleit.onmicrosoft.com/large-language-models/m6-inference-results".replace(
    "/dbfs", "dbfs:"
)
batch_inference_results.write.format("delta").mode("append").save(
    inference_results_path
)

And that's it!  To create a production job, we could for example take the new lines of code above, put them in a new notebook, and schedule it as an automated workflow.  MLflow can be integrated with essentially any deployment system, but for more information specific to this Databricks workspace, see the "Use model for inference" documentation for [AWS](https://docs.databricks.com/machine-learning/manage-model-lifecycle/index.html#use-model-for-inference), [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/manage-model-lifecycle/#--use-model-for-inference), or [GCP](https://docs.gcp.databricks.com/machine-learning/manage-model-lifecycle/index.html#use-model-for-inference).

We did not cover model serving for real-time inference, but MLflow models can be deployed to any cloud or on-prem serving systems.  For more information, see the [open-source MLflow Model Registry docs](https://mlflow.org/docs/latest/model-registry.html) or the [Databricks Model Serving docs](https://docs.databricks.com/machine-learning/model-serving/index.html).

For other topics not covered, see ["The Big Book of MLOps."](https://www.databricks.com/resources/ebook/the-big-book-of-mlops)

## Summary

We have now walked through a full example of going from development to production.  Our LLM pipeline was very simple, but LLM Ops for a more complex workflow (such as fine-tuning a custom model) would be very similar.  You still follow the basic Ops steps of:
* Development: Creating the pipeline or model, tracking the process in the MLflow Tracking server and saving the final pipeline or model.
* Staging: Registering a new model or version in the MLflow Model Registry, testing it, and promoting it through Staging to Production.
* Production: Creating an inference job, or creating a model serving endpoint.

-sandbox
&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>